# Entropy-Based Discretization

## Entropy
[A Simple Guide to Entropy-Based Discretization - Kevin Meurer](http://kevinmeurer.com/a-simple-guide-to-entropy-based-discretization/)

#### Calculation of entropy
$Entropy(D) = -\sum_{i = 1}^{m} p_i \cdot \log_2 p_i$

Where $D$ is our data. $m$ the number of classifier values we have and $p_i$ the probability for each classifier.

#### Calculation of net entropy for a split
$NetEntropy(D) = \cfrac{\left\vert{D_1}\right\vert}{\left\vert{D}\right\vert} \cdot Entropy(D_1) + \cfrac{\left\vert{D_2}\right\vert}{\left\vert{D}\right\vert} \cdot Entropy(D_2)$

The net entropy with splits is calculated by taking the entropy for each split and weighting it by it's share in the entire dataset.

#### Calculation of entropy gain for a split
$Gain(E_{new}) = E_{initial} - E_{new}$

Where $E_{inital}$ is the initial entropy of the set and $E_{new}$  is the new entropy with split.

## Example

In [167]:
from math import log

# Entropy calculator
def entropy(data):
    labels = [d[1] for d in data] # Get label for each data point
    distinct_labels = set(labels) # Get distinct labels in set
    
     # If there is only one label return entropy of 1 ( to avoid log_2(1) )
    if len(distinct_labels) == 1: 
        print("Total: %d\n%s:\n\tcount: %d\tprob: %.3f\nEntropy: %.3f" % 
              (len(data), distinct_labels.pop(), len(data), 1.0, 0.0))
        return 0.0

    probs = [ # Calculate probabilites for each distinct label
        labels.count(label)/len(data) # Count of each label by total number of occurrences
        for label in distinct_labels]
    
    # Some printing
    print("Total: %d" % len(data))
    for label in distinct_labels:
        count = labels.count(label)
        prob = count/len(data)
        bits = log(prob, 2)
        print("%s:\n\tcount: %d\tprob: %.3f\tlog_2(p): %.3f\tp*log:%.3f" % 
              (label, count, prob, bits, prob*bits))
    print("Entropy: %.3f" % -sum([p * log(p, 2) for p in probs]))
        
    # Calculate total entropy by weighting the number of bits necessary to
    # represent the probability ( log_2(probability) ) by the probability itself
    return -sum([p * log(p, 2) for p in probs])

In [168]:
data = [(0.1,   'FAIL'),
        (0.2,   'FAIL'),
        (0.8,   'FAIL'),
        (0.9,   'OK'),
        (1.0,   'FAIL'),
        (4.0,   'OK'),
        (10.0,  'OK'),
        (50.0,  'OK')]

total_entropy = entropy(data)
total_entropy

Total: 8
FAIL:
	count: 4	prob: 0.500	log_2(p): -1.000	p*log:-0.500
OK:
	count: 4	prob: 0.500	log_2(p): -1.000	p*log:-0.500
Entropy: 1.000


1.0

In [169]:
def entropy_gain(candidate_boundary):
    bin_1 = [d for d in data if d[0] < candidate_boundary]
    bin_2 = [d for d in data if d[0] > candidate_boundary]
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Bin 1:", bin_1)
    entropy_1 = entropy(bin_1)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Bin 2:", bin_2)
    entropy_2 = entropy(bin_2)
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    total = len(data)
    net_entropy = len(bin_1)/total * entropy_1 + len(bin_2)/total * entropy_2
    print("New net entropy:")
    print("(%d/%d) * %.3f + (%d/%d) * %.3f = %.3f" % 
          (len(bin_1), total, entropy_1, len(bin_2), total, entropy_2, net_entropy))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("Entropy gain:")
    print("%.3f = %.3f (initial) - %.3f (new)" % (total_entropy - net_entropy, total_entropy, net_entropy))

In [170]:
entropy_gain(0.5)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bin 1: [(0.1, 'FAIL'), (0.2, 'FAIL')]
Total: 2
FAIL:
	count: 2	prob: 1.000
Entropy: 0.000
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bin 2: [(0.8, 'FAIL'), (0.9, 'OK'), (1.0, 'FAIL'), (4.0, 'OK'), (10.0, 'OK'), (50.0, 'OK')]
Total: 6
FAIL:
	count: 2	prob: 0.333	log_2(p): -1.585	p*log:-0.528
OK:
	count: 4	prob: 0.667	log_2(p): -0.585	p*log:-0.390
Entropy: 0.918
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
New net entropy:
(2/8) * 0.000 + (6/8) * 0.918 = 0.689
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Entropy gain:
0.311 = 1.000 (initial) - 0.689 (new)


In [172]:
entropy_gain(0.95)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bin 1: [(0.1, 'FAIL'), (0.2, 'FAIL'), (0.8, 'FAIL'), (0.9, 'OK')]
Total: 4
FAIL:
	count: 3	prob: 0.750	log_2(p): -0.415	p*log:-0.311
OK:
	count: 1	prob: 0.250	log_2(p): -2.000	p*log:-0.500
Entropy: 0.811
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bin 2: [(1.0, 'FAIL'), (4.0, 'OK'), (10.0, 'OK'), (50.0, 'OK')]
Total: 4
FAIL:
	count: 1	prob: 0.250	log_2(p): -2.000	p*log:-0.500
OK:
	count: 3	prob: 0.750	log_2(p): -0.415	p*log:-0.311
Entropy: 0.811
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
New net entropy:
(4/8) * 0.811 + (4/8) * 0.811 = 0.811
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Entropy gain:
0.189 = 1.000 (initial) - 0.811 (new)


In [174]:
print_info(2.5)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bin 1: [(0.1, 'FAIL'), (0.2, 'FAIL'), (0.8, 'FAIL'), (0.9, 'OK'), (1.0, 'FAIL')]
Total: 5
FAIL:
	count: 4	prob: 0.800	log_2(p): -0.322	p*log:-0.258
OK:
	count: 1	prob: 0.200	log_2(p): -2.322	p*log:-0.464
Entropy: 0.722
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Bin 2: [(4.0, 'OK'), (10.0, 'OK'), (50.0, 'OK')]
Total: 3
OK:
	count: 3	prob: 1.000
Entropy: 0.000
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
New net entropy:
(5/8) * 0.722 + (3/8) * 0.000 = 0.451
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Entropy gain:
0.549 = 1.000 (initial) - 0.451 (new)
